### Лабораторная работа №2. Теория игр
Выполнили студенты: Петренко Людмила М33001, Кусайкина Елизавета М33001, Шалимов Иван М33021

In [39]:
import numpy as np
import matplotlib.pyplot as plt
import json
import simplex__lab_1 as simplex

In [75]:
class Game:
    def __init__(self):
        self.matrix: np.array()
        self.simplify_flag = 1
        self.gamer = 'a'
        self.beta = 0
        self.optimal_a = []
        self.optimal_b = []

    def load_data_from_json(self, filename: str) -> None:
        with open(filename) as f:
            data = json.load(f)
        self.gamer = data["gamer"]
        self.matrix = [[
            data['a ' + str(i)]['b ' + str(j)] for j in range(1, len(data['a ' + str(i)]) + 1)]
                for i in range(1, len(data))]
        self.original_matrix = np.copy(self.matrix)
        while self.simplify_flag:
            self.simplify_flag = self.simplify_matrix()

    def simplify_matrix(self, simplify_flag = 0):
        # удаление доминируемых стратегий игрока a
        for i in range(len(self.matrix)):
            for k in range(len(self.matrix)):
                if k == i:
                    continue
                if k >= len(self.matrix) or i >= len(self.matrix):
                    break
                number = 0
                for j in range(len(self.matrix[i])):
                    if self.matrix[i][j] <= self.matrix[k][j]:
                        number += 1
                if number == len(self.matrix[i]):
                    self.matrix = self.matrix[:i] + self.matrix[i + 1:]
                    simplify_flag = 1
        
        # удаление доминируемых стратегий игрока b
        for k in range(len(self.matrix[0])):
            for j in range(len(self.matrix[0])):
                number = 0
                if k == j:
                    continue
                if k >= len(self.matrix[0]) or j >= len(self.matrix[0]):
                    break
                for i in range(len(self.matrix)):
                    if self.matrix[i][j] <= self.matrix[i][k]:
                        number += 1
                if number == len(self.matrix):
                    for i in range(len(self.matrix)):
                        self.matrix[i] = self.matrix[i][:j] + self.matrix[i][j + 1:]
                    simplify_flag = 1
        return simplify_flag
    
    def solve(self):
        result = self.check_for_clean_strategies()
        if result is not None:
            if result < 0:
                if self.gamer == 'a':
                    self.gamer = 'b'
                else:
                    self.gamer = 'a'
                result = -result
            print(f"Решение в чистых стратегиях: \nигрок {self.gamer} победил, цена игры = {result}")
            print(f"Оптимальные чистые стратегии: \nпервый игрок = {self.optimal_a}, второй игрок = {self.optimal_b}")
        else:
            print("Оптимальные чистые стратегии: Нет таких")
    
        # решение прямой задачи
        self._make_matrix_positive()
        transponent_matrix = self.matrix.T
        
        constraints = [{' coefs ': transponent_matrix[i].tolist() , ' type ': 'gte', ' b ': 1} for i in range(len(transponent_matrix))]
        data = {' f ': [1 for i in range(len(transponent_matrix[0]))] ,
                ' goal ': 'min' ,
                ' constraints ': constraints}
 
        with open('simplex_data.json', 'w') as file:
            json.dump(data, file)

        task_x = simplex.LinearTask()
        task_x.load_data_from_json('simplex_data.json')
        teta_x, x_ = task_x.solve()

        # решение двойственной задачи
        constraints = [{' coefs ': self.matrix[i].tolist() , ' type ': 'lte' , ' b ': 1} for i in range(len(self.matrix))]
        data = {' f ': [1 for i in range(len(self.matrix[0]))] ,
                ' goal ': 'max' ,
                ' constraints ': constraints}
 
        with open('simplex_data.json', 'w') as file:
            json.dump(data, file)

        task_y = simplex.LinearTask()
        task_y.load_data_from_json('simplex_data.json')
        teta_y, y_ = task_y.solve()
        
        strategies_x = np.array(x_)/teta_x
        strategies_y = np.array(y_)/teta_y

        print(f'Стратегия первого игрока: {strategies_x},\n Стратегия второго игрока: {strategies_y}')

        print(f'Цена игры: {1/teta_x - self.beta}')

    
    def _make_matrix_positive(self):
        self.beta = np.amin(self.matrix)
        self.matrix = np.add(self.matrix, self.beta)

    def check_for_clean_strategies(self):
        if len(self.matrix) == 1 and len(self.matrix[0]) == 1:
           return self.matrix[0][0]
        
        row = []
        for j in range(len(self.matrix[0])):
            column = [self.matrix[i][j] for i in range(len(self.matrix))]
            i = column.index(max(column))
            row.append(max(column))
        j = row.index(min(row))
        self.minimax = self.matrix[i][j]

        column = []
        for i in range(len(self.matrix)):
            row = [self.matrix[i][j] for j in range(len(self.matrix[0]))]
            j = row.index(min(row))
            column.append(min(row))
        i = column.index(max(column))
        self.maximin = self.matrix[i][j]
        print("Maximin: ", self.maximin, "\nMinimax: ", self.minimax)

        if self.maximin == self.minimax:
            self.optimal_a, self.optimal_b = self.matrix[i], [self.matrix[i_][j] for i_ in range(len(self.matrix))]
            return self.minimax

#### Примеры результатов

In [66]:
game = Game()
game.load_data_from_json("example.json")
game.solve()

Maximin:  2 
Minimax:  3
Оптимальные чистые стратегии: Нет таких
Стратегия первого игрока: [0.2 0.8],
 Стратегия второго игрока: [0.6 0.4]
Цена игры: 2.4000000000000004


In [70]:
game = Game()
game.load_data_from_json("example1.json")
game.solve()

Maximin:  8 
Minimax:  7
Оптимальные чистые стратегии: Нет таких
Стратегия первого игрока: [0.         0.33333333 0.66666667],
 Стратегия второго игрока: [0.33333333 0.66666667]
Цена игры: 6.666666666666673


In [76]:
game = Game()
game.load_data_from_json("example2.json")
game.solve()

Maximin:  6 
Minimax:  6
Решение в чистых стратегиях: 
игрок a победил, цена игры = 6
Оптимальные чистые стратегии: 
первый игрок = [7, 6, 8, 9], второй игрок = [5, 4, 6]


('a', 6)